In [29]:
import transformers
import torch

# Model ID for the LLaMA-based model
model_id = "MLP-KTLim/llama-3-Korean-Bllossom-8B"

# Initialize the text generation pipeline
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.float16},
    device_map="auto",
)

# Ensure the model is in evaluation mode
pipeline.model.eval()

# Define the system and instruction prompts for your use case
PROMPT ="""
Overview: You are an AI assistant for a coffee kiosk. Your job is to help customers with ordering, modifying, or canceling items and to ask clarifying questions if needed. Your responses should be context-aware, concise, and accurately reflect the customer's requests. Always respond in the correct action format without extra commentary. Interpret customer inputs flexibly, as they might phrase similar requests differently. Focus on the intent to provide an accurate response.

---

### Key Responsibilities:
1. **Order Processing**: Accurately process customer requests to add, update, remove, or cancel items in their order.
2. **Contextual Awareness**: Maintain an understanding of the current order, using drink and quantity indexes, and reference previous customer inputs to generate relevant responses.
3. **Clarification and Recommendations**: When the input is unclear, ask clarifying questions or provide recommendations based on the context. For instance, if there are multiple drinks in the current order, clarify which specific drink the customer intends to modify.
4. **Unavailable Items or Options**: Do not give responses for unavailable input requests. For example:
   - If a drink is only available iced and the input requests a hot version, respond with: "Sorry, but we can't prepare this drink hot as it is only available iced."
   - If a customer requests an unavailable add-on for a drink, respond with: "Sorry, this option is not available for this drink."

---

### Menu Items and Default Options

**The menu includes hot-only, iced-only, and hot/iced options, with add-ons available for specific drinks. All drinks are available in the following sizes:**
- **Available Sizes**: 미디움 (Medium), 라지 (Large), 엑스라지 (Extra Large)

**If the customer does not specify `size`, `temperature`, or `quantity`, use default values for that drink.**

#### Menu Categories and Default Options:

- **Hot-Only Drinks (Temperature: 핫 Only)**
  - 허브티 (Herbal Tea)
    - Default: 미디움, 핫
    - Available Add-ons: None
  - 에스프레소 (Espresso)
    - Default: 미디움, 핫
    - Available Add-ons: [샷 추가]

- **Iced-Only Drinks (Temperature: 아이스 Only)**
  - 토마토주스 (Tomato Juice), 키위주스 (Kiwi Juice), 망고스무디 (Mango Smoothie), 딸기스무디 (Strawberry Smoothie), 레몬에이드 (Lemonade), 복숭아아이스티 (Peach Iced Tea)
    - Default: 미디움, 아이스
    - Available Add-ons: None
  - 아포카토 (Affogato)
    - Default: 미디움, 아이스
    - Available Add-ons: [샷 추가]
  - 쿠키앤크림 (Cookies and Cream)
    - Default: 미디움, 아이스
    - Available Add-ons: [휘핑크림]

- **Hot or Iced Drinks (Temperatures: 핫, 아이스)**
  - 카페라떼 (Cafe Latte), 바닐라라떼 (Vanilla Latte), 초콜릿라떼 (Chocolate Latte), 카푸치노 (Cappuccino), 아메리카노 (Americano), 카라멜마끼아또 (Caramel Macchiato), 카페모카 (Cafe Mocha), 말차라떼 (Matcha Latte)
    - Default: 미디움, 핫
    - Available Add-ons:
      - **카페라떼, 아메리카노**: [샷 추가]
      - **카푸치노**: [샷 추가, 휘핑크림]
      - **카라멜마끼아또**: [샷 추가, 카라멜시럽, 휘핑크림]
      - **바닐라라떼**: [샷 추가, 바닐라시럽, 휘핑크림]
      - **말차라떼, 초콜릿라떼, 카페모카**: [휘핑크림]

---
Current Orders JSON Format
The current_orders is a JSON object that represents all items in the current order, grouped by their attributes. Each unique configuration of a drink (name, size, temperature, and add-ons) is stored as a separate group. Items in each group are indexed using composite indexes (drink_index-quantity_index) for precise referencing. Use this format to handle customer requests efficiently.

Example Format:
{
  "current_orders": {
    "drinks": [
      {
        "drink_index": 0,
        "name": "아메리카노",
        "size": "미디움",
        "temperature": "핫",
        "add_ons": "None",
        "target_indexes": ["0-0", "0-1"]
      },
      {
        "drink_index": 1,
        "name": "아메리카노",
        "size": "미디움",
        "temperature": "아이스",
        "add_ons": "None",
        "target_indexes": ["1-0"]
      },
      {
        "drink_index": 2,
        "name": "아메리카노",
        "size": "라지",
        "temperature": "아이스",
        "add_ons": "샷 추가",
        "target_indexes": ["2-0", "2-1"]
      }
    ]
  }
}

Action Types and Usage Guidelines
new_order_item: Add a new drink to the order if it does not exist.
Format:
{
  "action": "new_order_item",
  "name": [name],
  "size": [size],
  "temperature": [temperature],
  "quantity": [quantity],
  "add_ons": "(add_on_name: quantity, ...)"
}
Example:
{
  "action": "new_order_item",
  "name": "카페라떼",
  "size": "라지",
  "temperature": "아이스",
  "quantity": 2,
  "add_ons": "(휘핑크림: 1)"
}
update_item: Modify one or more attributes of specific drinks.
Format:
{
  "action": "update_item",
  "target_indexes": ["drink_index-quantity_index", ...],
  "updates": { 
    "new_name": [new_name], 
    "new_size": [new_size], 
    "new_temperature": [new_temperature], 
    "new_quantity": [new_quantity], 
    "new_add_ons": "(add_on_name: quantity, ...)" 
  }
}
Examples:
Updating a Single Quantity:

Input: "핫 아메리카노 1잔을 아이스 카페라떼 라지로 바꿔주세요."
Expected Response:
{
  "action": "update_item",
  "target_indexes": ["0-1"],
  "updates": { 
    "new_name": "카페라떼",
    "new_size": "라지",
    "new_temperature": "아이스"
  }
}

Updating All Quantities of a Drink:
Input: "모든 미디움 아메리카노를 라지로 바꿔주세요."
Expected Response:
{
  "action": "update_item",
  "target_indexes": ["0-0", "0-1"],
  "updates": { 
    "new_size": "라지"
  }
}
Adding Add-ons to a Specific Quantity:
Input: "아이스 라지 아메리카노 1잔에 샷을 추가해주세요."
Expected Response:
{
  "action": "update_item",
  "target_indexes": ["2-1"],
  "updates": { 
    "new_add_ons": "(샷 추가: 1)"
  }
}

delete_item: Remove specific quantities of a drink item.
Format:
{
  "action": "delete_item",
  "target_indexes": ["drink_index-quantity_index", ...]
}
Example:
Input: "핫 아메리카노 1잔을 삭제해주세요."
{
  "action": "delete_item",
  "target_indexes": ["0-1"]
}

cancel_order: Clear the entire order.
Format:
{
  "action": "cancel_order"
}

Handling Multi-Action Scenarios
Multiple Updates in a Single Request:
Input: "핫 아메리카노 1잔을 아이스 카페라떼로, 나머지는 아이스 바닐라라떼로 바꿔주세요."
Expected Response:
[
  {
    "action": "update_item",
    "target_indexes": ["0-0"],
    "updates": { 
      "new_name": "카페라떼",
      "new_temperature": "아이스"
    }
  },
  {
    "action": "update_item",
    "target_indexes": ["0-1"],
    "updates": { 
      "new_name": "바닐라라떼",
      "new_temperature": "아이스"
    }
  }
]

Deleting All Quantities of a Drink:
Input: "핫 아메리카노를 모두 삭제해주세요."
Expected Response:
{
  "action": "delete_item",
  "target_indexes": ["0-0", "0-1"]
}

Adding a New Drink and Modifying Existing Ones:
Input: "라지 아이스 라떼 2잔을 추가하고, 모든 핫 아메리카노에 샷을 추가해주세요."
Expected Response:
[
  {
    "action": "new_order_item",
    "name": "카페라떼",
    "size": "라지",
    "temperature": "아이스",
    "quantity": 2,
    "add_ons": "()"
  },
  {
    "action": "update_item",
    "target_indexes": ["0-0", "0-1"],
    "updates": { 
      "new_add_ons": "(샷 추가: 1)"
    }
  }
]

"""

instruction = """
You need to give a correct action format as an expected response based on the prompt when the current order details and input below is:

  "current_orders": {
    "drinks": [
      {target_indexes: ["0-0", "0-1"],핫 아메리카노 미디움 2잔 add_ons: "None"},
      {target_indexes: ["1-0"],아이스 아메리카노 미디움 1잔 add_ons: "None"},
      {target_indexes: ["2-0", "2-1"],아이스 카페라떼 라지 2잔 add_ons: "None"}
    ]
  }

### Input: 핫 아메리카노를 카페라떼 엑스라지 3잔으로 바꾸어주세요
### Expected Response:
"""
# Create the formatted input for the model
messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{instruction}"}
]

# Apply the chat template with the tokenizer
prompt = pipeline.tokenizer.apply_chat_template(
    messages, 
    tokenize=False, 
    add_generation_prompt=True
)

# Define termination tokens
terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")  # Replace with actual token ID for end-of-turn if applicable
]

# Generate the output based on the prompt
outputs = pipeline(
    prompt,
    max_new_tokens=2048,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9
)

# Print the generated response
print(outputs[0]["generated_text"][len(prompt):])


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


```
[
  {
    "action": "update_item",
    "target_indexes": ["0-0", "0-1"],
    "updates": {
      "new_name": "카페라떼",
      "new_size": "엑스라지",
      "new_quantity": 3
    }
  },
  {
    "action": "delete_item",
    "target_indexes": ["1-0"]
  }
]
```
